In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt


#Potentially useful astropy stuff
import astropy
import astropy.io.ascii as ascii
from astropy.table import Table
from astropy.io import fits
from astropy.stats import LombScargle
from astropy.stats import sigma_clipped_stats
from astropy.stats import sigma_clip
from astropy.modeling import powerlaws
from astropy import constants as const
from astropy import units as u

#For reading in and organizing data
import pandas as pd
import requests
import json

#Misc
import scipy
import sklearn
from sklearn.preprocessing import scale

In [2]:
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.size'] = 12

In [3]:
# read data

data1=np.loadtxt('Data/phot211046195r2_ssc.2m0335.dat')
data2=np.loadtxt('Data/phot210327027r2_ssc.2m0355.dat')

time1,flux1,xx1,yy1=data1[:,0],data1[:,1],data1[:,2],data1[:,3]
time2,flux2,xx1,yy2=data2[:,0],data2[:,1],data2[:,2],data2[:,3]

In [4]:
def invgaussian(a ,m, s, x):
    g = -a * np.exp(-(m-x)**2 / s**2) + 1 
    return g

In [5]:
def bandpass_ifft(t, flux, low_cutoff, high_cutoff, sample=1, 
                  M=None, inv_box=False, gf_sig = 1, Filter='box', Plot='', tflare=None):
    """Bandpass filtering on a real signal using inverse FFT
    
    Inputs
    =======
    
    X: 1-D numpy array of floats, the real time domain signal (time series) to be filtered
    Low_cutoff: float, frequency components below this frequency will not pass the filter (physical frequency in unit of Hz)
    High_cutoff: float, frequency components above this frequency will not pass the filter (physical frequency in unit of Hz)
    sample: float, the sampling frequency of the signal (physical frequency in unit of Hz)    
    inv_box: If using box filter, setting inv=True filters out frequencies outside the box
    Filter: Default filter is box, can choose 'Gaussian' also
    
    Notes
    =====
    1. The input signal must be real, not imaginary nor complex
    2. The Filtered_signal will have only half of original amplitude. Use abs() to restore. 
    3. In Numpy/Scipy, the frequencies goes from 0 to F_sample/2 and then from negative F_sample to 0. 
    
    """        
    
    if not tflare is None:
        flare_ind = (np.abs(t-tflare)).argmin()
        plt.plot(t[flare_ind-40:flare_ind+40],flux[flare_ind-40:flare_ind+40])
        plt.show()
        
    #perform fft
    spectrum = np.fft.rfft(flux) 
    freq = np.fft.rfftfreq(len(flux), sample)
    freq_sort = np.sort(spectrum)
    
    

    #calculate the index of the cut off points
    lc = np.abs(freq) < Low_cutoff
    hc = np.abs(freq) > High_cutoff
    between = ~(lc + hc)
    
    ps = np.abs(spectrum)**2
    
    if ('PS' in Plot) or ('All' in Plot):
        plt.plot(freq, np.log10(ps))
        plt.title("power spectrum")
        plt.xlabel('Frequency (1/day)')
        plt.ylabel('Power Spectral Density')
        #plt.xlim(0,100)
        #plt.savefig('Figures/spec.png', bbox_inches='tight', pad_inches=0.5)
        plt.show()

    if ('DFT' in Plot) or ('All' in Plot):
        plt.plot(freq, spectrum)
        #plt.plot(freq[between], spectrum[between], alpha=0.5)
        plt.title("real fourier transform ")
        plt.xlabel('Frequency (1/day)')
        plt.ylabel('Amplitude')
        #plt.xlim(0,100)
        #plt.savefig('Figures/fft.png', bbox_inches='tight', pad_inches=0.5)
        plt.show()
    
    
    
    if Filter == 'box':
    
      #filtered_spectrum = spectrum.copy()
    
        if inv_box == True:
            x_1 = np.arange(0, Low_cutoff, 0.1)
            x_2 = np.arange(High_cutoff, np.max(freq), 0.1)
            plt.plot(freq, spectrum)
            plt.fill_between(x_1, [plt.ylim()[0]] * len(x_1), 
                     [plt.ylim()[1]] * len(x_1), color='r', alpha=0.3)
            plt.fill_between(x_2, [plt.ylim()[0]] * len(x_2), 
                     [plt.ylim()[1]] * len(x_2), color='r', alpha=0.3)
            plt.title("range to suppress")
            plt.figure()
            filtered_spectrum[lc] = 0.
            filtered_spectrum[hc] = 0.
        else:
            x_ = np.arange(Low_cutoff, High_cutoff, 0.1)
            plt.plot(freq, spectrum)
            plt.fill_between(x_, [plt.ylim()[0]] * len(x_), 
                     [plt.ylim()[1]] * len(x_), color='r', alpha=0.3)
            plt.title("range to suppress")
            plt.figure()
            filtered_spectrum[between] = 0.
    
        elif Filter == 'Gaussian':
            ig = invgaussian(1, np.median([low_cutoff,high_cutoff]), gf_sig, freq)
            filtered_spectrum = spectrum * ig
            if ('filter' in Plot) or ('All' in Plot):
                plt.plot(freq, ig)
                plt.title('Gaussian Filter')
                #plt.savefig('Figures/gfilter.png')
                #plt.xlim(0,100)
                plt.figure()
        
        
    else:
        filtered_spectrum = spectrum

    if ('spec_filtered' in Plot) or ('All' in Plot):
        plt.plot(freq, filtered_spectrum, label="filtered spectrum")
        plt.plot(freq, spectrum, c='k', ls="--", label="spectrum", alpha=0.5)
        plt.title("Unfiltered vs. Filtered Spectrum")
        plt.xlabel('Frequency (1/day)')
        plt.ylabel('Amplitude')
        ldg = plt.legend(fontsize=12)
        #plt.xlim(0,100)
        #plt.savefig('Figures/filter_compare.png', bbox_inches='tight', pad_inches=0.5)
        plt.figure()

    filtered_signal = np.fft.irfft(filtered_spectrum)# Construct filtered signal
    
    if not tflare is None:
        plt.plot(t[flare_ind-40:flare_ind+40],filtered_signal[flare_ind-40:flare_ind+40])
        plt.show()

    if ('signal_filtered' in Plot) or ('All' in Plot):
        fig = plt.figure(figsize=(15,10)) 
        plt.plot(t, filtered_signal, label="filtered signal")
        plt.plot(t, flux, c='k', ls="--", label="original signal", alpha=0.5)
        plt.xlabel('Time')
        plt.ylabel('Amplitude')
        plt.title("Unfiltered vs. Filtered Signal")
        #plt.savefig('Figures/filtered_signal.png', bbox_inches='tight', pad_inches=0.5)
        plt.legend()
        #Filtered_signal = np.zeros_like(Filtered_signal)
    return spectrum, freq, filtered_spectrum, filtered_signal, Low_cutoff, High_cutoff

In [6]:
flux1norm = flux1.copy() / sigma_clip(flux1, sigma=3).mean()

In [7]:
flux2norm = flux2 / sigma_clip(flux2, sigma=3).mean()

In [ ]:
Low_cutoff, High_cutoff, F_sample = 4.0, 4.2, np.diff(time1).mean()
Spectrum1, frequency1, Filtered_spectrum1, Filtered_signal1, Low_freq, High_freq = bandpass_ifft(
    time1, flux1norm, Low_cutoff, High_cutoff, 
                          F_sample, gf_sig = 0.2, Filter='', Plot='PS', tflare=2253.65)

In [ ]:
Low_cutoff, High_cutoff, F_sample = 4.0, 4.2, np.diff(time2).mean()
Spectrum2, frequency2, Filtered_spectrum2, Filtered_signal2, Low_freq, High_freq = bandpass_ifft(time2, flux2norm, Low_cutoff, High_cutoff, 
                          F_sample, gf_sig = 0.2, Filter='', Plot='PS', tflare=2258.59)